# Blockchain Main Classes

In [3]:
import hashlib
import pickle
import copy
import timeit

## Completar la clase RSA_KEY

In [4]:
class rsa_key:            
    def __init__(self,bits_modulo=2048,e=2**16+1):
        def generatePQ(NdeBits):
            r1=randint(0,2^(NdeBits/2));
            r2=randint(0,2^(NdeBits/2));
            p=next_probable_prime(r1);
            while gcd(e,p-1)!=1:
                print( ".");
                p=next_probable_prime(p+2);

            q=next_probable_prime(r2);
            while gcd(e,q-1)!=1:
                print( "+");
                q=next_probable_prime(q+2);
            return p, q

        
        self.primeP, self.primeQ = generatePQ(bits_modulo)
        self.publicExponent = e
        self.privateExponent = mod(e^-1, (self.primeP-1)*(self.primeQ-1))
        self.modulus = self.primeP * self.primeQ
        self.privateExponentModulusPhiP = mod(self.privateExponent, self.primeP-1)
        self.privateExponentModulusPhiQ = mod(self.privateExponent, self.primeQ-1)
        self.inverseQModulusP = mod(self.primeQ^-1, self.primeP)
        
    def sign(self,message):
        message = mod(message, self.modulus)
        mp=Integer(mod(message^self.privateExponentModulusPhiP,self.primeP));
        mq=Integer(mod(message^self.privateExponentModulusPhiQ,self.primeQ));
        h=mod((mp-mq)*Integer(self.inverseQModulusP),self.modulus);
        m_tcr=mod((mq+self.primeQ*h),self.modulus);
        return m_tcr
    
    def sign_slow(self,message):
        return pow(message,self.privateExponent,self.modulus)

## Completar la clase RSA_PUBLIC_KEY

In [5]:
class rsa_public_key:
    def __init__(self, rsa_key):
        self.publicExponent = rsa_key.publicExponent
        self.modulus = rsa_key.modulus
    def verify(self, message, signature):
        return message == pow(signature,self.publicExponent,self.modulus)

## Completar la clase TRANSACTION

In [6]:
class transaction:
    def __init__(self, message, RSAkey):
        self.public_key = rsa_public_key(RSAkey)
        self.message = message
        self.signature = RSAkey.sign(message)
    def verify(self):
        return self.public_key.verify(self.message, self.signature)

## Completar la clase BLOCK

In [7]:
class block:
    def __init__(self):
        self.block_hash = 0
        self.previous_block_hash = 0
        self.transaction = transaction(0, rsa_key())
        self.seed = -1
        self.rango = 2^32
        
    def genesis(self,transaction):
        d = 16
        previous_block_hash = 0
        valido = False
        while (not valido) and self.seed < self.rango:
            self.seed += 1
            entrada=str(previous_block_hash)
            entrada=entrada+str(transaction.public_key.publicExponent)
            entrada=entrada+str(transaction.public_key.modulus)
            entrada=entrada+str(transaction.message)
            entrada=entrada+str(transaction.signature)
            entrada=entrada+str(self.seed)
            h=int(hashlib.sha256(entrada.encode()).hexdigest(),d)
            valido = h < (2^(256-d))
                    
        self.previous_block_hash = 0
        self.block_hash = h
        self.transaction = transaction
        
    def next_block(self, transaction):
        d = 16
        previous_block_hash = self.block_hash
        self.seed = -1
        valido = False
        while not valido and self.seed < self.rango:
            self.seed += 1
            entrada=str(previous_block_hash)
            entrada=entrada+str(transaction.public_key.publicExponent)
            entrada=entrada+str(transaction.public_key.modulus)
            entrada=entrada+str(transaction.message)
            entrada=entrada+str(transaction.signature)
            entrada=entrada+str(self.seed)
            h=int(hashlib.sha256(entrada.encode()).hexdigest(),d)
            valido = h < (2^(256-d))
        self.previous_block_hash = self.block_hash
        self.block_hash = h
        self.transaction = transaction
        
    def verify_block(self):
        d = 16
        condicion1 = self.transaction.verify()
        condicion2 = self.block_hash < (2^(256-d))
        condicion3 = self.previous_block_hash < (2^(256-d))
        
        return condicion1 and condicion2 and condicion3

## Completar la clase BLOCK_CHAIN

In [8]:
class block_chain:
    def __init__(self,transaction):
        first_block = block()
        first_block.genesis(transaction)
        self.list_of_blocks = [first_block]
        
    def add_block(self,transaction):
        ultimo = copy.deepcopy(self.list_of_blocks[-1])
        ultimo.next_block(transaction)
        self.list_of_blocks.append(ultimo)
        
    def verify(self):
        d = 16
        bien = True
        index = 0
        
        bien = self.list_of_blocks[0].previous_block_hash == 0
        if not bien:
            return bien
        
        while index < len(self.list_of_blocks) and bien:
            actual = self.list_of_blocks[index]
            bien = actual.verify_block()
            index += 1
        
        if not bien:
            return bien
        
        index = 0
        while index < len(self.list_of_blocks) - 1 and bien:
            index += 1
            bien = self.list_of_blocks[index-1].block_hash == self.list_of_blocks[index].previous_block_hash
        
        if not bien:
            return bien
        
        index = -1
        while index < len(self.list_of_blocks) - 1 and bien:
            index += 1
            
            previous_block_hash = self.list_of_blocks[index].previous_block_hash
            transaction = self.list_of_blocks[index].transaction
            seed = self.list_of_blocks[index].seed
            
            entrada=str(previous_block_hash)
            entrada=entrada+str(transaction.public_key.publicExponent)
            entrada=entrada+str(transaction.public_key.modulus)
            entrada=entrada+str(transaction.message)
            entrada=entrada+str(transaction.signature)
            entrada=entrada+str(seed)
            h=int(hashlib.sha256(entrada.encode()).hexdigest(),d)
            
            bien = h == self.list_of_blocks[index].block_hash
        
        return bien

# Comprobación de la implementación

In [9]:
fichero = 'Cadena_bloques_valida.block'
lectura = open(fichero, 'rb')
cadenaBloques = pickle.load(lectura)
lectura.close()

In [10]:
print(cadenaBloques.verify())

True


In [11]:
fichero = 'Cadena_bloques_bloque_falso.block'
lectura = open(fichero, 'rb')
cadenaBloques = pickle.load(lectura)
lectura.close()

In [12]:
print(cadenaBloques.verify())

False


In [32]:
fichero = 'Cadena_bloques_seed_falsa.block'
lectura = open(fichero, 'rb')
cadenaBloques = pickle.load(lectura)
lectura.close()

In [33]:
print(cadenaBloques.verify())

False


In [34]:
fichero = 'Cadena_bloques_transaccion_falsa.block'
lectura = open(fichero, 'rb')
cadenaBloques = pickle.load(lectura)
lectura.close()

In [35]:
print(cadenaBloques.verify())

False


# Tablas comparación firmado con TCR y sin TCR

# Cadena válida de 100 bloques

In [13]:
inicio = timeit.default_timer()

transaccion = transaction(0, rsa_key())
bloques = block_chain(transaccion)

for i in range(1, 100):
    transaccion = transaction(i, rsa_key())
    bloques.add_block(transaccion)

final = timeit.default_timer()

print(final - inicio)

256.4997470680005


In [38]:
bloques.verify()

True

In [39]:
with open('Cadena_Pablo_Vega.block', 'wb') as file:
    pickle.dump(bloques, file)

# Cadena 100 bloques válida hasta la 41